# **3. Procesamiento** **del** **dataset** **input** **para** **la** **segunda** **fase** **de** **entrenamiento** **NER**

En el siguiente módulo se preprocesa el dataset utilizado para la primera iteración de entrenamiento del modelo NER, utilizando las conclusiones obtenidas deeste para mejorar el dataset de entrenamiento y conseguir de este modo un modelo más robusto. La primera etapa, consistente en cargar el dataset y obtener mediante un pipeline un dataset limpio para usar en el entrenemiento, es idéntico al del módulo de preprocesamiento (módulo 1)

In [ ]:
!pip install requests beautifulsoup4 pandas
!pip install -q openai ipywidgets python-dotenv
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.6 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import os
import json

In [ ]:
from pathlib import Path
# Leerlo de vuelta
contenido = Path("recetas.json").read_text(encoding="utf-8")
recetas_cargadas = json.loads(contenido)

In [ ]:
#Funcion para añadir resumen a la receta
import re
from typing import Dict, List, Any

def agregar_resumen_a_receta(receta: Dict[str, Any], clave_pasos: str = "pasos", clave_resumen: str = "resumen") -> Dict[str, Any]:
    """
    Añade receta[clave_resumen] uniendo receta[clave_pasos] (lista de strings)
    en una sola cadena, separadas por punto. Limpia puntuación final duplicada.

    - Si no hay pasos o no es una lista de strings, pone resumen = "".
    - Devuelve el propio diccionario (mutado) por conveniencia.
    """
    pasos = receta.get(clave_pasos) or []
    if not isinstance(pasos, list):
        receta[clave_resumen] = ""
        return receta

    limpios: List[str] = []
    for p in pasos:
        if not isinstance(p, str):
            continue
        s = p.strip()
        # quita puntuación final para evitar ".."
        s = re.sub(r"\s*([.;:!?…]+)\s*$", "", s)
        if s:
            limpios.append(s)

    # une con ". " y añade punto final si hay contenido
    resumen = ". ".join(limpios)
    if resumen:
        resumen += "."
    receta[clave_resumen] = resumen
    return receta

In [ ]:
for receta in recetas_cargadas:
  receta = agregar_resumen_a_receta(receta)

In [ ]:
#Normalización del dataset

import re, unicodedata
from typing import Tuple

_STOPWORDS = {"de","del","la","el","las","los","al","y","en","con","para"}

def _quitar_tildes(txt: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFD", txt) if unicodedata.category(c) != "Mn")

def _build_norm_index_map(original: str) -> Tuple[str, List[int]]:
    """
    Devuelve (texto_normalizado, idx_map) donde idx_map[i] es el índice del
    carácter original correspondiente al carácter normalizado i.
    """
    norm_chars, idx_map = [], []
    for i, ch in enumerate(original):
        base = _quitar_tildes(ch).lower()
        for _ in base:
            norm_chars.append(_)
            idx_map.append(i)
    return "".join(norm_chars), idx_map

def _plural_es_basico(pal: str) -> str:
    w = _quitar_tildes(pal.lower())
    if not w:
        return w
    if w.endswith("z"):
        return w[:-1] + "ces"
    if w.endswith(("s","x")):
        return w  # heurística: la mayoría quedan invariables
    if re.search(r"[aeiou]$", w):
        return w + "s"
    return w + "es"

def _plural_de_frase(ing: str) -> str:
    toks = ing.lower().split()
    if not toks:
        return ing.lower()
    idx = len(toks) - 1
    while idx >= 0 and toks[idx] in _STOPWORDS:
        idx -= 1
    if idx < 0:
        return ing.lower()
    toks[idx] = _plural_es_basico(toks[idx])
    return " ".join(toks)

def asociar_ingredientes_en_resumen(
    receta: Dict[str, Any],
    usar_limites_palabra: bool = True,
    detectar_plural: bool = True
) -> Dict[str, Any]:
    """
    Muy simple: si un ingrediente (en singular) está en receta['resumen'],
    lo asocia. También captura el plural (heurístico) y es insensible a tildes.
    Añade:
      - receta['ingredientes_en_resumen'] (lista deduplicada)
      - receta['spans_resumen'] con {ingredient, start, end, surface}
    Offsets 0-based sobre el resumen original. 'end' exclusivo.
    """
    resumen = (receta.get("resumen") or "")
    ingredientes = receta.get("ingredientes") or []
    if not isinstance(resumen, str) or not isinstance(ingredientes, list):
        receta["ingredientes_en_resumen"] = []
        receta["spans_resumen"] = []
        return receta

    # Texto normalizado + mapa a original (insensible a tildes)
    resumen_norm, idx_map = _build_norm_index_map(resumen)

    presentes: List[str] = []
    spans: List[Dict[str, Any]] = []

    for ing in ingredientes:
        ing_sing = (ing or "").strip()
        if not ing_sing:
            continue

        # variantes: singular + plural (heurístico)
        variantes = [ing_sing]
        if detectar_plural:
            pl = _plural_de_frase(ing_sing)
            if pl and pl != ing_sing:
                variantes.append(pl)

        # buscamos cada variante sobre el texto normalizado
        seen_positions = set()
        for var in variantes:
            var_norm = _quitar_tildes(var).lower()
            if not var_norm:
                continue

            if usar_limites_palabra:
                pat = re.compile(rf"(?<!\w){re.escape(var_norm)}(?!\w)")
                matches = list(pat.finditer(resumen_norm))
                if matches and ing_sing not in presentes:
                    presentes.append(ing_sing)
                for m in matches:
                    s_n, e_n = m.span()
                    s = idx_map[s_n]
                    e = idx_map[e_n - 1] + 1
                    if (s, e) in seen_positions:
                        continue
                    seen_positions.add((s, e))
                    spans.append({
                        "ingredient": ing_sing,
                        "start": s,
                        "end": e,
                        "surface": resumen[s:e]
                    })
            else:
                start_n = 0
                found = False
                while True:
                    idx_n = resumen_norm.find(var_norm, start_n)
                    if idx_n == -1:
                        break
                    s = idx_map[idx_n]
                    e = idx_map[idx_n + len(var_norm) - 1] + 1
                    if (s, e) not in seen_positions:
                        seen_positions.add((s, e))
                        spans.append({
                            "ingredient": ing_sing,
                            "start": s,
                            "end": e,
                            "surface": resumen[s:e]
                        })
                        found = True
                    start_n = idx_n + len(var_norm)
                if found and ing_sing not in presentes:
                    presentes.append(ing_sing)

    # deduplicar presentes manteniendo orden
    seen_ing = set()
    presentes = [x for x in presentes if not (x in seen_ing or seen_ing.add(x))]

    receta["ingredientes_en_resumen"] = presentes
    receta["spans_resumen"] = sorted(spans, key=lambda s: (s["start"], s["end"]))
    return receta


In [ ]:
for receta in recetas_cargadas:
  asociar_ingredientes_en_resumen(receta,usar_limites_palabra=True)

In [ ]:
#Obtención de la lista de los ingredientes usados en el primer entrenamiento NER (con más de 50 aparaciones)
def top_ingredientes_frecuentes(
    recetas: List[Dict[str, Any]],
    umbral: int = 50,
    clave_spans: str = "spans_resumen",
    clave_ing: str = "ingredient"
) -> List[str]:
    """
    Cuenta menciones por ingrediente a partir de 'spans_resumen' y
    devuelve una lista de ingredientes (strings) que superan el umbral.
    - 'umbral' aplica como '>' (estrictamente mayor que).
    """
    c = Counter()
    for r in recetas:
        for s in (r.get(clave_spans) or []):
            ing = s.get(clave_ing)
            if ing:
                c[ing] += 1

    # filtrar > umbral y ordenar desc por conteo, luego alfabético
    filtrados = [(ing, n) for ing, n in c.items() if n > umbral]
    ordenados = sorted(filtrados, key=lambda x: (-x[1], x[0]))
    return [ing for ing, _ in ordenados]

In [ ]:
ingredientes_frecuentes = top_ingredientes_frecuentes(recetas_cargadas)
print(ingredientes_frecuentes)

['sal', 'ajo', 'cebolla', 'patata', 'aceite de oliva', 'huevo', 'tomate', 'pollo', 'harina', 'agua', 'perejil', 'arroz', 'leche', 'mantequilla', 'zanahoria', 'azúcar', 'pimentón', 'aceite', 'puerro', 'pimiento', 'bacalao', 'vino blanco', 'nata', 'pan', 'salmón', 'caldo de pollo', 'cebolleta', 'vinagre', 'caldo', 'pasta', 'queso', 'limón', 'pimienta', 'caldo de pescado', 'merluza', 'calabacín', 'garbanzo', 'atún']


Procesamiento del dataset para el segundo entrenamiento: se eliminan aquellos ingredientes que el modelo malinterpreta y se simplifican los ingredientes que tienen un significado parecido o idéntico solapándolos.



Eliminamos de la lista los ingredientes identificados como problemáticos en la primera iteración de entrenamiento:

- **cebolleta**: al tokenizar el dataset se obtiene el lexema ceboll-, que coincide con el de cebolla, lo cual hace que el modelo erre al tratar de identificarlo.

- **pimienta**: al tokenizar el dataset se obtiene el lexema pimient-, que coincide con el de pimiento, lo cual hace que el modelo erre al tratar de identificarlo.

In [ ]:
ingredientes_frecuentes.remove('cebolleta')
ingredientes_frecuentes.remove('pimienta')
print(ingredientes_frecuentes)

['sal', 'ajo', 'cebolla', 'patata', 'aceite de oliva', 'huevo', 'tomate', 'pollo', 'harina', 'agua', 'perejil', 'arroz', 'leche', 'mantequilla', 'zanahoria', 'azúcar', 'pimentón', 'aceite', 'puerro', 'pimiento', 'bacalao', 'vino blanco', 'nata', 'pan', 'salmón', 'caldo de pollo', 'vinagre', 'caldo', 'pasta', 'queso', 'limón', 'caldo de pescado', 'merluza', 'calabacín', 'garbanzo', 'atún']


Filtramos las recetas de manera que el span solo se haga con los ingredientes de la lista top_frecuentes. Si una receta no contiene ninguno de esos ingredientes se elimina

In [ ]:
from collections.abc import Iterable
from typing import List, Dict, Any

def filtrar_spans_por_frecuentes(
    recetas: List[Dict[str, Any]],
    ingredientes_frecuentes: Iterable,  # ["tomate", ...] o [("tomate", 123), ...]
    clave_spans: str = "spans_resumen",
    clave_ing: str = "ingredient",
    actualizar_ingredientes_en_resumen: bool = True,
    eliminar_recetas_sin_spans: bool = False,
    eliminar_si_ingredientes_en_resumen_vacio: bool = True
) -> List[Dict[str, Any]]:
    """
    Mantiene en cada receta solo los spans cuyo 'ingredient' esté en 'ingredientes_frecuentes'.
    - Si 'actualizar_ingredientes_en_resumen' es True, recalcula esa lista con los spans filtrados.
    - Si 'eliminar_recetas_sin_spans' es True, descarta recetas sin spans tras filtrar.
    - Si 'eliminar_si_ingredientes_en_resumen_vacio' es True, descarta recetas cuyo
      'ingredientes_en_resumen' quede vacío o nulo tras el filtrado.
    Devuelve una **nueva lista** (no modifica la original).
    """
    # Normaliza lista de permitidos a set de strings
    allowed = set()
    for x in ingredientes_frecuentes or []:
        name = x[0] if isinstance(x, (list, tuple)) else x
        if isinstance(name, str) and name.strip():
            allowed.add(name.strip())

    nuevas_recetas: List[Dict[str, Any]] = []
    for r in recetas:
        spans = r.get(clave_spans) or []
        filtrados = [s for s in spans
                     if isinstance(s, dict) and s.get(clave_ing) in allowed]

        # Clona receta y escribe spans filtrados
        r2 = dict(r)
        r2[clave_spans] = sorted(
            filtrados, key=lambda s: (s.get("start", 0), s.get("end", 0))
        )

        # Actualiza lista de ingredientes_en_resumen a partir de los spans filtrados
        if actualizar_ingredientes_en_resumen:
            vistos = set()
            ing_list = []
            for s in r2[clave_spans]:
                lab = s.get(clave_ing)
                if lab and lab not in vistos:
                    vistos.add(lab)
                    ing_list.append(lab)
            r2["ingredientes_en_resumen"] = ing_list

        # Eliminaciones opcionales
        if eliminar_recetas_sin_spans and not r2[clave_spans]:
            continue

        if eliminar_si_ingredientes_en_resumen_vacio:
            ing_res = r2.get("ingredientes_en_resumen")
            if not ing_res:  # None, [], "", etc. se consideran vacíos
                continue

        nuevas_recetas.append(r2)

    return nuevas_recetas

In [ ]:
recetas_filtradas = filtrar_spans_por_frecuentes(recetas_cargadas, ingredientes_frecuentes=ingredientes_frecuentes)

Para la simplificación de los ingredientes repetidos/parecidos, realizamos un remapeo de las etiquetas:

- 'aceite de oliva' -> 'aceite'
- 'caldo de pollo' -> 'caldo'
- 'caldo de pescado' -> 'caldo

Se utilizan las funciones:

- ***_norm***: normaliza una cadena quitando tildes, pasando a minúsculas, cambiando “_” por espacio y colapsando/recortando espacios.

- ***_dedup_preservando_orden***: elimina duplicados (y elementos vacíos) de una secuencia de strings preservando el primer orden de aparición.

- ***remap_labels_en_recetas***: aplica un mapa de reemplazos de labels  sobre varios los campos span e ingredeinetes de cada receta, deduplica listas y cuenta los cambios y devuelve (nuevas_recetas, num_reemplazos).



In [ ]:
# Función para remapear
import unicodedata
from typing import  Tuple, Iterable, Union

def _norm(s: str) -> str:
    # minúsculas, sin tildes, "_"→" ", colapsa espacios
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")
    s = s.lower().replace("_", " ").strip()
    return " ".join(s.split())

def _dedup_preservando_orden(xs: Iterable[str]) -> List[str]:
    seen, out = set(), []
    for x in xs:
        if x and x not in seen:
            seen.add(x); out.append(x)
    return out

def remap_labels_en_recetas(
    recetas: List[Dict[str, Any]],
    reemplazos: Union[Dict[str, str], Tuple[str, str], List[Tuple[str, str]]],
    clave_spans: str = "spans_resumen",
    clave_ing: str = "ingredient",
    tocar_ingredientes_en_resumen: bool = True,
    tocar_ingredientes: bool = True,
) -> Tuple[List[Dict[str, Any]], int]:
    """
    Aplica remapeos de labels (p.ej., {'aceite de oliva':'aceite', 'pimentón dulce':'pimentón'})
    sobre TODAS las recetas en: spans_resumen, ingredientes_en_resumen e ingredientes.

    Devuelve (nuevas_recetas, num_reemplazos).
    """

    # Normaliza 'reemplazos' a dict
    if isinstance(reemplazos, tuple) and len(reemplazos) == 2:
        reemplazos = {reemplazos[0]: reemplazos[1]}
    elif isinstance(reemplazos, list):
        reemplazos = {a: b for (a, b) in reemplazos}

    if not isinstance(reemplazos, dict):
        raise ValueError("Argumento 'reemplazos' debe ser dict, (old,new) o lista de tuplas [(old,new), ...].")

    # Mapa normalizado (insensible a tildes/case/_)
    norm_map: Dict[str, str] = {}
    for src, dst in reemplazos.items():
        if isinstance(src, str) and isinstance(dst, str):
            norm_map[_norm(src)] = dst.strip()

    cambios = 0
    nuevas: List[Dict[str, Any]] = []

    for r in recetas:
        r2 = dict(r)

        # 1) spans_resumen
        spans = r2.get(clave_spans) or []
        nuevos_spans = []
        for s in spans:
            if not isinstance(s, dict):
                continue
            s2 = dict(s)
            lab = s2.get(clave_ing)
            key = _norm(lab) if isinstance(lab, str) else None
            if key in norm_map:
                s2[clave_ing] = norm_map[key]
                cambios += 1
            nuevos_spans.append(s2)
        r2[clave_spans] = sorted(nuevos_spans, key=lambda x: (x.get("start", 0), x.get("end", 0)))

        # 2) ingredientes_en_resumen
        if tocar_ingredientes_en_resumen and "ingredientes_en_resumen" in r2:
            lst = r2.get("ingredientes_en_resumen") or []
            lst2 = []
            for lab in lst:
                key = _norm(lab) if isinstance(lab, str) else None
                lst2.append(norm_map.get(key, lab))
                if key in norm_map:
                    cambios += 1
            r2["ingredientes_en_resumen"] = _dedup_preservando_orden(lst2)

        # 3) ingredientes (lista canónica)
        if tocar_ingredientes and "ingredientes" in r2:
            lst = r2.get("ingredientes") or []
            lst2 = []
            for lab in lst:
                key = _norm(lab) if isinstance(lab, str) else None
                lst2.append(norm_map.get(key, lab))
                if key in norm_map:
                    cambios += 1
            r2["ingredientes"] = _dedup_preservando_orden(lst2)

        nuevas.append(r2)

    return nuevas, cambios

In [ ]:
mapa = {
    "aceite de oliva": "aceite",
    "caldo de pescado": "caldo",
    "caldo de pollo": "caldo",
}
recetas_remap, n = remap_labels_en_recetas(recetas_filtradas, mapa)
print("Reemplazos:", n)

Reemplazos: 1546


A modo de ejemplo se muestra una entrada de la lista, correspondiente a la receta "Ensalada de pasta", una vez aplicado el remapeo de ingredientes parecidos (en este caso aceite de oliva) + eliminación de ingredientes conflictivos (en este caso pimienta)

In [ ]:
recetas_remap[241]

{'titulo': 'Ensalada de pasta, un clásica receta de verano',
 'url': 'https://recetasdecocina.elmundo.es/2021/07/ensalada-pasta-receta-verano.html',
 'ingredientes': ['pasta',
  'maíz',
  'atún',
  'huevo',
  'pimiento',
  'canónigo',
  'sal',
  'pimienta',
  'aceite',
  'vinagre',
  'comino',
  'orégano'],
 'pasos': ['Comenzamos cociendo la pasta en abundante agua con un poquito de sal como indique el fabricante.',
  'Una vez tengamos la pasta cocida. La pasamos por agua fría y así cortamos la cocción para que tengamos la pasta fría y no templada.',
  'Ahora por otra parte cocemos los huevos en agua durante 10 minutos.',
  'Seguidamente picamos el resto de ingredientes que haya que picar y sacamos de la lata el maíz, el atún etc…',
  'Mezclamos muy bien todo y aliñamos con aceite de oliva y vinagre. Ponemos las especias que más nos gusten.'],
 'resumen': 'Comenzamos cociendo la pasta en abundante agua con un poquito de sal como indique el fabricante. Una vez tengamos la pasta cocida. 

Ahora que ya tenemos las recetas filtradas, reestructuramos los diccionarios para darles un formato adecuado para el entrenamiento de NER

In [ ]:
def _resolve_overlaps_longest(spans):
    if not spans: return []
    spans = sorted(spans, key=lambda s: (s["start"], -(s["end"]-s["start"])))
    out = []
    for s in spans:
        if not out or s["start"] >= out[-1]["end"]:
            out.append(s)
        # si solapa, se queda el previo (más largo, ya que ordenamos por longitud desc)
    return out

def recetas_a_ner_multiclase(recetas, resolver_solapes=True):
    """
    Usa el campo 'ingredient' de cada span como label (ya normalizado).
    Formato de salida: [{id, text, entities:[{start,end,label}, ...]}, ...]
    """
    ejemplos = []
    label_set = set()

    for i, r in enumerate(recetas, 1):
        text = r.get("resumen") or ""
        spans = r.get("spans_resumen") or []
        use = _resolve_overlaps_longest(spans) if resolver_solapes else sorted(spans, key=lambda s:(s["start"], s["end"]))

        entities = []
        for s in use:
            start, end = int(s["start"]), int(s["end"])
            # validación suave de offsets
            surface = text[start:end]
            if "surface" in s and s["surface"] != surface:
                # si no cuadra, puedes loguear o continuar
                pass
            lab = s.get("ingredient", "")
            if not lab:
                continue
            entities.append({"start": start, "end": end, "label": lab})
            label_set.add(lab)

        ejemplos.append({
            "id": r.get("id", f"receta_{i:05d}"),
            "text": text,
            "entities": entities
        })

    return ejemplos, sorted(label_set)

In [ ]:
train_data, labels = recetas_a_ner_multiclase(recetas_remap)

In [ ]:
#guardamos el dataset que servirá de entrenamiento (+ validacion + test) del modelo NER
Path("train_data_2.json").write_text(
    json.dumps(train_data, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

1283963

In [ ]:
# Leerlo de vuelta para chequear que todo esta bien
contenido = Path("train_data_2.json").read_text(encoding="utf-8")
data_train = json.loads(contenido)

In [ ]:
data_train[922]

{'id': 'receta_00923',
 'text': 'En un bol preparamos las hamburguesas, mezclamos la carne, con el huevo, el orégano, el ajo picado, el pan rallado, la mostaza en grano, sal y pimienta. Ahora como veis en la foto ponéis la carne picada macerada en una tabla y aplastáis con la mano haciendo la forma redonda. Con la ayuda de un molde hacemos un agujero y retiramos la carne. Mirad que bien quedan 🙂. Las ponemos en la plancha por una parte. Una vez que le damos la vuelta las ponemos un poco más lentas y ponemos el huevo frito dentro. Podéis taparlas así el huevo frito se hará con el calor más rápidamente.',
 'entities': [{'start': 66, 'end': 71, 'label': 'huevo'},
  {'start': 88, 'end': 91, 'label': 'ajo'},
  {'start': 137, 'end': 140, 'label': 'sal'},
  {'start': 483, 'end': 488, 'label': 'huevo'},
  {'start': 526, 'end': 531, 'label': 'huevo'}]}